# Mnist

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'Mnist'
# dataset = 'IllusionAnimals'

df = pd.read_csv(f'Gemini_{dataset}_results.csv')

In [23]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,ill_images,illusion_images_filtered,illusionless_images,illusionless_images_filtered,raw_images
0,Mnist_1,A field of blooming sunflowers swaying in the ...,low quality,1.5,7,'digit 3','digit 7','No illusion','No illusion','digit 7'
1,Mnist_2,A peaceful countryside scene with grazing shee...,low quality,1.5,2,'No illusion','digit 2','No illusion','No illusion',The correct class is 'digit 2'.
2,Mnist_3,A tranquil pond with lily pads floating on the...,low quality,1.5,1,'No illusion','digit 7','No illusion','No illusion','digit 1'
3,Mnist_4,A sunny vineyard with rows of ripe grapes,low quality,1.5,0,'No illusion','digit 8','No illusion','No illusion',The correct class is 'digit 0'.
4,Mnist_5,A picturesque vineyard at sunset with the sky ...,low quality,1.5,4,'No illusion','digit 4','No illusion','No illusion','digit 4'
...,...,...,...,...,...,...,...,...,...,...
1104,Mnist_1105,Misty jungle surrounded by vibrant flowers and...,low quality,1.5,8,'No illusion','digit 9','No illusion','No illusion','digit 9'
1105,Mnist_1106,A forest with blooming flowers,low quality,1.5,0,'No illusion','digit 0','No illusion','No illusion',The correct class is 'digit 0'.
1106,Mnist_1107,"Desolate desert landscape, shifting sands illu...",low quality,1.5,5,'No illusion','digit 5','No illusion','No illusion','digit 5'
1107,Mnist_1108,A vast desert with a towering canyon in the di...,low quality,1.5,0,'No illusion','No illusion','No illusion','No illusion',The correct class is 'digit 0'.


In [24]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [25]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}

# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_ill_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusion_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}.issubset({item.strip().lower() for item in raw_classes[dataset]})

In [26]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'capital letter z',
 'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'letter s',
 'the class of the object in the picture is digit 1.',
 'the class of the object in the picture is digit 7.',
 'the class of the picture is digit 0.',
 'the class of the picture is digit 1.',
 'the class of the picture is digit 2.',
 'the class of the picture is digit 3.',
 'the class of the picture is digit 4.',
 'the class of the picture is digit 5.',
 'the class of the picture is digit 6.',
 'the class of the picture is digit 7.',
 'the class of the picture is digit 9.',
 'the correct class is digit 0.',
 'the correct class is digit 1.',
 'the correct class is digit 2.',
 'the correct class is digit 3.',
 'the correct class is digit 4.',
 'the correct class is digit 5.',
 'the correct class is digit 7.',
 'the correct class is digit 8.',
 'the correct class is digit 9.'}

In [27]:
df[f'raw_images'].unique()

array([" 'digit 7'", " The correct class is 'digit 2'.", " 'digit 1'",
       " The correct class is 'digit 0'.", " 'digit 4'", " 'digit 9'",
       " 'digit 5'", ' digit 6',
       " The class of the picture is 'digit 0'.", ' digit 5',
       " 'digit 3'", ' digit 4', " 'digit 6'",
       " The correct class is 'digit 3'.", ' digit 2',
       " The class of the picture is 'digit 1'.", " 'digit 2'",
       ' digit 3', " The class of the picture is 'digit 5'.",
       " The correct class is 'digit 5'.", ' digit 9',
       " The class of the picture is 'digit 7'.", " 'digit 8'",
       " The correct class is 'digit 8'.",
       " The class of the object in the picture is 'digit 1'.",
       " The class of the picture is 'digit 3'.", " 'capital letter z'",
       ' digit 8', " 'digit 0'", " The correct class is 'digit 9'.",
       ' digit 7', " The correct class is 'digit 7'.",
       " The correct class is 'digit 4'.",
       " The class of the picture is 'digit 9'.",
       " The class 

In [28]:
raw_classes[dataset]

['digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9']

In [29]:
def find_class(x, classes):
    for item in classes:
        if item in x:
            return item
    return x

In [30]:
# df['raw_images'] = df['raw_images'].apply(lambda x: x if x in raw_classes[dataset] else x)
df['raw_images'] = df['raw_images'].apply(lambda x: find_class(x, raw_classes[dataset]))

In [31]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'capital letter z',
 'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'letter s'}

In [32]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'capital letter z',
 'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'letter s'}

## Raw

In [33]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = df['label'].apply(lambda x: x.strip().lower())
else:
    y_true = df['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df[f'raw_images'].apply(lambda x: x.strip().replace('\'', '').lower())

In [34]:
set(list(y_pred))

{'capital letter z',
 'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'letter s'}

In [35]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.8935978358881875
Precision: 0.7623450638582546
Recall: 0.7426516618445741
F1 score: 0.7445538396409276


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 89.36
Precision: 76.23
Recall: 74.27
F1 score: 74.46


## Illusion

In [37]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))

# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))

y_pred = list(df[f'ill_images'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [38]:
set(y_pred)

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'no illusion'}

In [39]:
set(y_true)

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'no illusion'}

In [40]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.21821164889253486
Precision: 0.7273829233964512
Recall: 0.2214738668588824
F1 score: 0.23364542154276483


In [41]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 21.82
Precision: 72.74
Recall: 22.15
F1 score: 23.36


## Illusion Filtered

In [42]:
# {item.strip().replace('\'', '').lower() for item in df[f''].unique()}

In [43]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


y_pred = list(df[f'illusion_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [44]:
set(y_pred)

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'no illusion'}

In [45]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.638228055783429
Precision: 0.8344094660968646
Recall: 0.6437498464604294
F1 score: 0.6822655991370202


In [46]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 63.82
Precision: 83.44
Recall: 64.37
F1 score: 68.23


# FashionMnist

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'FashionMnist'
# dataset = 'IllusionAnimals'

df = pd.read_csv(f'Gemini_{dataset}_results.csv')

In [48]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,ill_images,illusion_images_filtered,illusionless_images,illusionless_images_filtered,raw_images
0,FashionMnist_1,Two friends hiking on a mountain during sunset,low quality,1.5,9,'No illusion',Sneaker,Trouser,'No illusion',Ankle boot
1,FashionMnist_2,A starry night sky with constellations glowing,low quality,1.5,2,'No illusion','No illusion','No illusion','No illusion','Pullover'
2,FashionMnist_3,A Thanksgiving feast giving thanks and love,low quality,1.5,1,Trouser,Trouser,No illusion,'No illusion',Trouser
3,FashionMnist_4,A gecko on a large leaf in a tropical rainforest,low quality,1.5,1,'No illusion','No illusion','No illusion','No illusion',Trouser
4,FashionMnist_5,A Hanukkah menorah lighting symbolizing faith,low quality,1.5,6,Coat,Coat,No illusion,'No illusion',Coat
...,...,...,...,...,...,...,...,...,...,...
1147,FashionMnist_1148,"Electrified waterfalls cascade, neon rivers re...",low quality,1.5,2,'No illusion',Coat,'No illusion','No illusion',The correct class is 'Pullover'.
1148,FashionMnist_1149,A rocky desert landscape with cacti and tumble...,low quality,1.5,2,'No illusion','T-shirt/top','No illusion','No illusion','Pullover'
1149,FashionMnist_1150,A majestic waterfall plunging into a crystal-c...,low quality,1.5,5,'No illusion','No illusion','No illusion','No illusion',Sandal
1150,FashionMnist_1151,"Eerie silence surrounds snowy mountains, glowi...",low quality,1.5,3,No illusion,'Coat','No illusion','No illusion',Dress


In [49]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [50]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}


In [51]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'ankle boot',
 'bag',
 'camera',
 'coat',
 'dress',
 'engine',
 'fish',
 'hat',
 'helmet',
 'kimono',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'the class of the object in the picture is bag.',
 'the class of the object in the picture is coat.',
 'the class of the object in the picture is dress.',
 'the class of the object in the picture is pullover.',
 'the class of the picture is bag.',
 'the class of the picture is pullover.',
 'the class of the picture is t-shirt/top.',
 'the correct class is bag.',
 'the correct class is bird.',
 'the correct class is bodysuit.',
 'the correct class is coat.',
 'the correct class is dress.',
 'the correct class is pullover.',
 'the correct class is sandal.',
 'the correct class is shirt.',
 'the correct class is sneaker.',
 'the correct class is t-shirt/top.',
 'trouser'}

In [52]:
raw_classes[dataset]

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [53]:
def find_class(x, classes):
    for item in classes:
        if item.lower() in x.lower():
            return item
    return x

In [54]:
# df['raw_images'] = df['raw_images'].apply(lambda x: x if x in raw_classes[dataset] else x)
df['raw_images'] = df['raw_images'].apply(lambda x: find_class(x, raw_classes[dataset]))

In [55]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'ankle boot',
 'bag',
 'camera',
 'coat',
 'dress',
 'engine',
 'fish',
 'hat',
 'helmet',
 'kimono',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'the correct class is bird.',
 'the correct class is bodysuit.',
 'trouser'}

In [56]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'ankle boot',
 'bag',
 'camera',
 'coat',
 'dress',
 'engine',
 'fish',
 'hat',
 'helmet',
 'kimono',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'the correct class is bird.',
 'the correct class is bodysuit.',
 'trouser'}

## Raw

In [57]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = df['label'].apply(lambda x: x.strip().lower())
else:
    y_true = df['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df[f'raw_images'].apply(lambda x: x.strip().replace('\'', '').lower())

In [58]:
set(list(y_pred))

{'ankle boot',
 'bag',
 'camera',
 'coat',
 'dress',
 'engine',
 'fish',
 'hat',
 'helmet',
 'kimono',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'the correct class is bird.',
 'the correct class is bodysuit.',
 'trouser'}

In [59]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.6727430555555556
Precision: 0.4056070039428924
Recall: 0.3743022151226429
F1 score: 0.3674093877383753


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 67.27
Precision: 40.56
Recall: 37.43
F1 score: 36.74


## Illusion

In [61]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))

# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))

# y_pred = list(df[f'ill_images'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df[f'illusionless_images'].apply(lambda x: x.strip().replace('\'', '').lower()))
y_pred = list(df[f'ill_images'].apply(lambda x: find_class(x, classes[dataset]).strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images'].apply(lambda x: find_class(x, classes[dataset]).strip().replace('\'', '').lower()))

In [62]:
classes[dataset]

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot',
 'no illusion']

In [63]:
set(y_pred)

{'ankle boot',
 'bag',
 'car',
 'coat',
 'dress',
 'hat',
 'no illusion',
 'pullover',
 'sandal',
 'saree',
 'shorts',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [64]:
set(y_true)

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'no illusion',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [65]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.11049723756906077
Precision: 0.3324963586582777
Recall: 0.08122773499390586
F1 score: 0.042431462976543065


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 11.05
Precision: 33.25
Recall: 8.12
F1 score: 4.24


## Illusion Filtered

In [67]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


y_pred = list(df[f'illusion_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images_filtered'].apply(lambda x: x.strip().replace('\'', '').lower()))

In [68]:
set(y_pred)

{'ankle boot',
 'bag',
 'car',
 'coat',
 'dress',
 'hat',
 'kimono',
 'no illusion',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [69]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.40568271507498027
Precision: 0.5420145110588435
Recall: 0.31975999279646
F1 score: 0.33262922044423393


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 40.57
Precision: 54.2
Recall: 31.98
F1 score: 33.26


# IllusionAnimals

In [71]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
# dataset = 'FashionMnist'
dataset = 'IllusionAnimals'

df = pd.read_csv(f'Gemini_{dataset}_results.csv')

In [72]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,ill_images,illusion_images_filtered,illusionless_images,illusionless_images_filtered,raw_images
0,IllusionAnimals_1,A raging river flowing through a dense jungle ...,low quality,2.5,cat,'No illusion','dog','No illusion','No illusion',The class of the animal in the picture is 'cat'.
1,IllusionAnimals_2,A starry night sky over a tranquil lake,low quality,2.5,cat,'No illusion','cat','No illusion','No illusion',The class of the animal in the picture is 'cat'.
2,IllusionAnimals_3,Jaguar (Panthera Onca) patrolling South Americ...,low quality,2.5,cat,'jaguar','cat','jaguar','cat',The class of the animal in the picture is 'cat'.
3,IllusionAnimals_4,"Flowing lava illuminates cavern walls, ancient...",low quality,2.5,cat,'No illusion','cat','No illusion','No illusion',The class of the animal in the picture is 'cat'.
4,IllusionAnimals_5,"Dense forest canopy, sunlight filters through,...",low quality,2.5,cat,cat,'cat','No illusion','No illusion',The class of the animal in the picture is 'cat'.
...,...,...,...,...,...,...,...,...,...,...
995,IllusionAnimals_996,Saharan sandstorm turning daytime into darkness,low quality,2.5,rooster,'rooster','rooster','No illusion','No illusion','rooster'
996,IllusionAnimals_997,Urban city with skyscrapers and traffic,low quality,2.5,rooster,Rooster,rooster,'No illusion','No illusion','rooster'
997,IllusionAnimals_998,A serene meadow with wildflowers and butterflies,low quality,2.5,rooster,'butterfly',rooster,'No illusion','No illusion','rooster'
998,IllusionAnimals_999,Elk bugling in crisp autumn air,low quality,2.5,rooster,'No illusion',rooster,'deer','deer','rooster'


In [73]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [74]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}


In [75]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'butterfly',
 'elephant',
 'fish',
 'horse',
 'insect',
 'pigeon',
 'rooster',
 'snake',
 'the class of the animal in the picture is butterfly.',
 'the class of the animal in the picture is cat.',
 'the class of the animal in the picture is deer.',
 'the class of the animal in the picture is dog.',
 'the class of the animal in the picture is elephant.',
 'the class of the animal in the picture is fish.',
 'the class of the animal in the picture is horse.',
 'the class of the animal in the picture is insect.',
 'the class of the animal in the picture is pigeon.',
 'the class of the animal in the picture is reptile.',
 'the class of the animal in the picture is rooster.',
 'the class of the animal in the picture is snake.',
 'the class of the object in the picture is butterfly.',
 'the class of the object in the picture is cat.',
 'the class of the object in the picture is deer.',
 'the class of the object in the picture is dog.',
 'the class of the object in the picture is fish.',
 'th

In [76]:
raw_classes[dataset]

['cat',
 'dog',
 'pigeon',
 'butterfly',
 'elephant',
 'horse',
 'deer',
 'snake',
 'fish',
 'rooster']

In [77]:
def find_class(x, classes):
    for item in classes:
        if item.lower() in x.lower():
            return item
    return x

In [78]:
# df['raw_images'] = df['raw_images'].apply(lambda x: x if x in raw_classes[dataset] else x)
df['raw_images'] = df['raw_images'].apply(lambda x: find_class(x, raw_classes[dataset]))

In [79]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'insect',
 'pigeon',
 'rooster',
 'snake',
 'the class of the animal in the picture is insect.',
 'the class of the animal in the picture is reptile.'}

In [80]:
{item.strip().replace('\'', '').lower() for item in df[f'raw_images'].unique()}

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'insect',
 'pigeon',
 'rooster',
 'snake',
 'the class of the animal in the picture is insect.',
 'the class of the animal in the picture is reptile.'}

## Raw

In [81]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = df['label'].apply(lambda x: x.strip().lower())
else:
    y_true = df['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df[f'raw_images'].apply(lambda x: x.strip().replace('\'', '').lower())

In [82]:
set(list(y_pred))

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'insect',
 'pigeon',
 'rooster',
 'snake',
 'the class of the animal in the picture is insect.',
 'the class of the animal in the picture is reptile.'}

In [83]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.872
Precision: 0.7692307692307693
Recall: 0.6707692307692308
F1 score: 0.6868181502327845


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 87.2
Precision: 76.92
Recall: 67.08
F1 score: 68.68


## Illusion

In [85]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))

# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))

# y_pred = list(df[f'ill_images'].apply(lambda x: x.strip().replace('\'', '').lower())) + list(df[f'illusionless_images'].apply(lambda x: x.strip().replace('\'', '').lower()))
y_pred = list(df[f'ill_images'].apply(lambda x: find_class(x, classes[dataset]).strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images'].apply(lambda x: find_class(x, classes[dataset]).strip().replace('\'', '').lower()))

In [86]:
classes[dataset]

['cat',
 'dog',
 'pigeon',
 'butterfly',
 'elephant',
 'horse',
 'deer',
 'snake',
 'fish',
 'rooster',
 'no illusion']

In [87]:
set(y_pred)

{'arctic fox',
 'bear',
 'bird',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'fox',
 'horse',
 'jaguar',
 'kangaroo',
 'leopard',
 'lion',
 'no illusion',
 'owl',
 'peacock',
 'pigeon',
 'polar bear',
 'rooster',
 'sloth',
 'snake',
 'squirrel',
 'the correct class is bear.',
 'the correct class is giraffe.',
 'the correct class is hyena.',
 'the correct class is kangaroo.',
 'the correct class is lion.',
 'the correct class is manatee.',
 'the correct class is octopus.',
 'the correct class is puffin.',
 'the correct class is shark.',
 'the correct class is tiger.',
 'the correct class is turtle.',
 'tractor',
 'turtle',
 'wolf'}

In [88]:
set(y_true)

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [89]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.25272727272727274
Precision: 0.19949109415687355
Recall: 0.07315789473684212
F1 score: 0.08120655677432137


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 25.27
Precision: 19.95
Recall: 7.32
F1 score: 8.12


## Illusion Filtered

In [91]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


y_pred = list(df[f'illusion_images_filtered'].apply(lambda x: find_class(x, classes[dataset]).strip().replace('\'', '').lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'illusionless_images_filtered'].apply(lambda x: find_class(x, classes[dataset]).strip().replace('\'', '').lower()))

In [92]:
set(y_pred)

{'arctic fox',
 'bear',
 'butterfly',
 'cat',
 'deer',
 'dog',
 'dolphin',
 'elephant',
 'fish',
 'horse',
 'kangaroo',
 'no illusion',
 'pigeon',
 'rooster',
 'sloth',
 'snake',
 'sphinx'}

In [93]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.8572727272727273
Precision: 0.5992051803565452
Recall: 0.5547058823529412
F1 score: 0.5639924534528766


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [94]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 85.73
Precision: 59.92
Recall: 55.47
F1 score: 56.4
